### combine, clean and format algorithm based scraped data with the API scrape data togather into 1 dataset

In [1]:
import pandas as pd
import numpy as np
import glob
import os

In [2]:
# list_of_csvs_ppt = sorted(glob.glob('scraping_data/ppt*.csv'))
list_of_csvs_API = sorted(glob.glob('scraping_data/video*.csv'))
list_of_csvs_Algo = glob.glob('scraping_data/Targeted_Ouput/*.csv')

In [3]:
extracted_text_df = pd.read_csv('scraping_data/Targeted_Ouput/1_transcaript_txt_sentences_level_corrected_timestamp.csv').drop(['Unnamed: 0'], axis = 1)

print(extracted_text_df.columns)
extracted_text_df.head()


Index(['file_name', 'raw_txt', 'raw_sentences', 'corrected_sentences',
       'corrected_txt', 'sentences_levenshtain_scores',
       'sentences_jaccard_scores', 'sentences_num_words_diff',
       'sentences_num_words_diff_abs', 'raw_sentences_word_counts',
       'correction_ratio', 'final_corrected_version_sentences_txt',
       'final_corrected_version_txt', 'cleaned_raw_sentences',
       'tokenized_sentences', 'sentence_level_timstamp_min_sec',
       'sentence_level_timstamp_max_sec', 'sentence_level_timstamp_min_minute',
       'sentence_level_timstamp_max_minute', 'audio_source', 'diff_ratio',
       'info_loss_perc', 'main_topics', 'meta_file', 'reference_text'],
      dtype='object')


,file_name,raw_txt,raw_sentences,corrected_sentences,corrected_txt,sentences_levenshtain_scores,sentences_jaccard_scores,sentences_num_words_diff,sentences_num_words_diff_abs,raw_sentences_word_counts,...,sentence_level_timstamp_min_sec,sentence_level_timstamp_max_sec,sentence_level_timstamp_min_minute,sentence_level_timstamp_max_minute,audio_source,diff_ratio,info_loss_perc,main_topics,meta_file,reference_text
0,insects_intro_ant_v,Did you find an ant hill. The app that you see...,Did you find an ant hill.,Did you find an old hill?,Did you find an old hill? The app only detects...,4.0,0.200000,0,0,7,...,11.666667,20.0,0.1944,0.3333,insects_intro_ant_v,0.721154,27.885,insect_ant,insects_intro_ant_t,['Ants are one of the most common insects that...
1,insects_intro_ant_v,Did you find an ant hill. The app that you see...,The app that you see outside or only a small n...,The app only detects a small number of the ant...,Did you find an old hill? The app only detects...,37.0,0.045455,6,6,20,...,23.333333,40.0,0.3889,0.6667,insects_intro_ant_v,0.721154,27.885,insect_ant,insects_intro_ant_t,['Ants are one of the most common insects that...
2,insects_intro_ant_v,Did you find an ant hill. The app that you see...,These huge groups of ants are called calm he's...,"These huge groups of ants are called ""Calmette...",Did you find an old hill? The app only detects...,49.0,0.161290,2,2,54,...,40.476190,60.0,0.6746,1.0000,insects_intro_ant_v,0.721154,27.885,insect_ant,insects_intro_ant_t,['Ants are one of the most common insects that...
3,insects_intro_ant_v,Did you find an ant hill. The app that you see...,All of the worker ants all females.,All of the workers are all females.,Did you find an old hill? The app only detects...,4.0,0.062500,0,0,8,...,59.411765,62.0,0.9902,1.0333,insects_intro_ant_v,0.721154,27.885,insect_ant,insects_intro_ant_t,['Ants are one of the most common insects that...
4,insects_intro_ant_v,Did you find an ant hill. The app that you see...,The meal ask job is the fertilized the queen t...,The meal ask job is the fertilized queen they ...,Did you find an old hill? The app only detects...,4.0,0.000000,1,1,24,...,62.400000,71.5,1.0400,1.1917,insects_intro_ant_v,0.721154,27.885,insect_ant,insects_intro_ant_t,['Ants are one of the most common insects that...


In [4]:
refernece_txt_df = extracted_text_df[['main_topics', 'reference_text']].drop_duplicates()

In [5]:
sorted(list(extracted_text_df['main_topics'].unique()))

['insect_ant', 'insect_beetle','bird_flamingo', 'bird_peacock', 'bird_tailorbird' ]

['insect_ant',
 'insect_beetle',
 'bird_flamingo',
 'bird_peacock',
 'bird_tailorbird']

In [6]:
## 1. read each API scraped video
## 2. calculate the end time stamp in each sentences
## 3. label each sentences with the main topic
## 4. combin each sentences togather into a whole text transcript with "." as sperators
## 5. create a new column called "transcript_text" to contain the whole transcriop text in each senetcnal level rows
## 6. finally put each df everything toagther into whole big DF

API_main_df = pd.DataFrame()

main_topics_lst = ['insect_ant', 'insect_beetle','bird_flamingo', 'bird_peacock', 'bird_tailorbird' ]

for idx in range(0, len(list_of_csvs_API)):
    
    api_temp = pd.read_csv(list_of_csvs_API[idx])
    
    # calculate the end timestamp
    api_temp['end'] = api_temp['start'] + api_temp['duration']
    
    # label each sentences with the main topic
    api_temp['main_topics'] = main_topics_lst[idx]
    
    # combine each sentences togather into a whole text transcript with "." as sperators
    video_ids = list(api_temp['Video ID'].unique())
    trans_temp_df_dict = {"Video_ID" : [], "Transcript_Txt": []}
    video_trans_df = pd.DataFrame()
        
    for video in video_ids:
        video_sentences = list(api_temp[api_temp['Video ID'] == video]['text'])
        transcript = ""

        for sentence in video_sentences:
            
            if video_sentences.index(sentence) == 0:
                transcript = transcript + sentence
            else:
                transcript = transcript + ". " + sentence
        trans_temp_df_dict["Video_ID"].append(video)
        trans_temp_df_dict["Transcript_Txt"].append(transcript)
        
    
    video_trans_df = pd.DataFrame.from_dict(trans_temp_df_dict)
    
    # 5. create a new column called "transcript_text" to contain the whole transcriop text in each senetcnal level rows
    api_temp = pd.merge(api_temp, video_trans_df, 'left', left_on = "Video ID" , right_on = "Video_ID")
    
    # finally put each df everything toagther into whole big DF
    API_main_df = pd.concat([API_main_df, api_temp], axis = 0).drop(['Unnamed: 0','Video_ID' ], axis = 1)
    

# add refenrece text
API_main_df = pd.merge(API_main_df,refernece_txt_df, 'left', left_on = 'main_topics' , right_on = 'main_topics')

# rename and reorder some of the API_name dataframe

API_main_df_col = ['text', 'start', 'duration', 'Video ID', 'Title', 'Age Restricted',
       'Length (min)', 'Views (thous)', 'Category', 'end', 'main_topics',
       'Transcript_Txt', 'reference_text']

API_main_df_new = ['final_corrected_version_sentences_txt', 'sentence_level_timstamp_min_sec', 'duration', 'Video_ID', 'Title', 'Age_Restricted',
       'Length_(min)', 'Views_(thous)', 'Category', 'sentence_level_timstamp_max_sec', 'main_topics',
       'final_corrected_version_txt', 'reference_text']

change_col_name_map = dict(zip(API_main_df_col, API_main_df_new))
API_main_df = API_main_df.rename(columns=change_col_name_map)

API_main_df['sentence_level_timstamp_min_minute'] = np.round(API_main_df['sentence_level_timstamp_min_sec']/60, 6)

API_main_df['sentence_level_timstamp_max_minute'] = np.round(API_main_df['sentence_level_timstamp_max_sec']/60, 6)

API_main_df = API_main_df[['Title', 'Video_ID', 'Category', 'Age_Restricted', \
             'final_corrected_version_sentences_txt', 'final_corrected_version_txt',\
             'sentence_level_timstamp_min_sec', 'sentence_level_timstamp_max_sec',\
             'sentence_level_timstamp_min_minute', 'sentence_level_timstamp_max_minute', \
             'duration', 'Length_(min)', 'Views_(thous)', \
            'main_topics', 'reference_text']]


In [7]:
print(API_main_df.columns)

API_main_df.head()

Index(['Title', 'Video_ID', 'Category', 'Age_Restricted',
       'final_corrected_version_sentences_txt', 'final_corrected_version_txt',
       'sentence_level_timstamp_min_sec', 'sentence_level_timstamp_max_sec',
       'sentence_level_timstamp_min_minute',
       'sentence_level_timstamp_max_minute', 'duration', 'Length_(min)',
       'Views_(thous)', 'main_topics', 'reference_text'],
      dtype='object')


,Title,Video_ID,Category,Age_Restricted,final_corrected_version_sentences_txt,final_corrected_version_txt,sentence_level_timstamp_min_sec,sentence_level_timstamp_max_sec,sentence_level_timstamp_min_minute,sentence_level_timstamp_max_minute,duration,Length_(min),Views_(thous),main_topics,reference_text
0,Children Learn About The Ant,cXUCUvcscXs,Ant,False,welcome to my tree house come and look,welcome to my tree house come and look. at thi...,0.560,4.080,0.009333,0.068000,3.520,5.333333,921.491,insect_ant,['Ants are one of the most common insects that...
1,Children Learn About The Ant,cXUCUvcscXs,Ant,False,at this,welcome to my tree house come and look. at thi...,2.560,6.080,0.042667,0.101333,3.520,5.333333,921.491,insect_ant,['Ants are one of the most common insects that...
2,Children Learn About The Ant,cXUCUvcscXs,Ant,False,this is my ant farm,welcome to my tree house come and look. at thi...,4.080,8.639,0.068000,0.143983,4.559,5.333333,921.491,insect_ant,['Ants are one of the most common insects that...
3,Children Learn About The Ant,cXUCUvcscXs,Ant,False,ants are amazing creatures lots of,welcome to my tree house come and look. at thi...,6.080,10.559,0.101333,0.175983,4.479,5.333333,921.491,insect_ant,['Ants are one of the most common insects that...
4,Children Learn About The Ant,cXUCUvcscXs,Ant,False,insects live by themselves and have to,welcome to my tree house come and look. at thi...,8.639,12.880,0.143983,0.214667,4.241,5.333333,921.491,insect_ant,['Ants are one of the most common insects that...


In [8]:
API_main_df.columns

Index(['Title', 'Video_ID', 'Category', 'Age_Restricted',
       'final_corrected_version_sentences_txt', 'final_corrected_version_txt',
       'sentence_level_timstamp_min_sec', 'sentence_level_timstamp_max_sec',
       'sentence_level_timstamp_min_minute',
       'sentence_level_timstamp_max_minute', 'duration', 'Length_(min)',
       'Views_(thous)', 'main_topics', 'reference_text'],
      dtype='object')

In [9]:
# rename some of the columns in extracted_text_df
extracted_text_df_processed = extracted_text_df.copy()
extracted_text_df_processed['Video_ID'] = extracted_text_df_processed['file_name']
extracted_text_df_processed['Views_(thous)'] = 0
extracted_text_df_processed['Category'] = ''
extracted_text_df_processed['duration'] = 0
extracted_text_df_processed['Length_(min)'] = 0
extracted_text_df_processed['Age_Restricted'] = False
extracted_text_df_processed['Title'] = extracted_text_df_processed['file_name']

extracted_text_df_processed = extracted_text_df_processed[[
    'Title', 'Video_ID', 'Category', 'Age_Restricted',\
    'final_corrected_version_sentences_txt', 'final_corrected_version_txt',\
       'sentence_level_timstamp_min_sec','sentence_level_timstamp_max_sec', \
    'sentence_level_timstamp_min_minute','sentence_level_timstamp_max_minute',\
    'duration', 'Length_(min)', 'Views_(thous)', 'main_topics','reference_text'
    
]]

In [10]:
extracted_text_df_processed.columns
extracted_text_df_processed.head()

,Title,Video_ID,Category,Age_Restricted,final_corrected_version_sentences_txt,final_corrected_version_txt,sentence_level_timstamp_min_sec,sentence_level_timstamp_max_sec,sentence_level_timstamp_min_minute,sentence_level_timstamp_max_minute,duration,Length_(min),Views_(thous),main_topics,reference_text
0,insects_intro_ant_v,insects_intro_ant_v,,False,Did you find an ant hill.,Did you find an ant hill. The app that you see...,11.666667,20.0,0.1944,0.3333,0,0,0,insect_ant,['Ants are one of the most common insects that...
1,insects_intro_ant_v,insects_intro_ant_v,,False,The app that you see outside or only a small n...,Did you find an ant hill. The app that you see...,23.333333,40.0,0.3889,0.6667,0,0,0,insect_ant,['Ants are one of the most common insects that...
2,insects_intro_ant_v,insects_intro_ant_v,,False,These huge groups of ants are called calm he's...,Did you find an ant hill. The app that you see...,40.476190,60.0,0.6746,1.0000,0,0,0,insect_ant,['Ants are one of the most common insects that...
3,insects_intro_ant_v,insects_intro_ant_v,,False,All of the worker ants all females.,Did you find an ant hill. The app that you see...,59.411765,62.0,0.9902,1.0333,0,0,0,insect_ant,['Ants are one of the most common insects that...
4,insects_intro_ant_v,insects_intro_ant_v,,False,The meal ask job is the fertilized the queen t...,Did you find an ant hill. The app that you see...,62.400000,71.5,1.0400,1.1917,0,0,0,insect_ant,['Ants are one of the most common insects that...


In [11]:
## combine and concat the API downlaod text with the algo scraped text into a final DF to be later feed into the models
main_df = pd.concat([API_main_df, extracted_text_df_processed ], axis = 0)
main_df.head()

,Title,Video_ID,Category,Age_Restricted,final_corrected_version_sentences_txt,final_corrected_version_txt,sentence_level_timstamp_min_sec,sentence_level_timstamp_max_sec,sentence_level_timstamp_min_minute,sentence_level_timstamp_max_minute,duration,Length_(min),Views_(thous),main_topics,reference_text
0,Children Learn About The Ant,cXUCUvcscXs,Ant,False,welcome to my tree house come and look,welcome to my tree house come and look. at thi...,0.560,4.080,0.009333,0.068000,3.520,5.333333,921.491,insect_ant,['Ants are one of the most common insects that...
1,Children Learn About The Ant,cXUCUvcscXs,Ant,False,at this,welcome to my tree house come and look. at thi...,2.560,6.080,0.042667,0.101333,3.520,5.333333,921.491,insect_ant,['Ants are one of the most common insects that...
2,Children Learn About The Ant,cXUCUvcscXs,Ant,False,this is my ant farm,welcome to my tree house come and look. at thi...,4.080,8.639,0.068000,0.143983,4.559,5.333333,921.491,insect_ant,['Ants are one of the most common insects that...
3,Children Learn About The Ant,cXUCUvcscXs,Ant,False,ants are amazing creatures lots of,welcome to my tree house come and look. at thi...,6.080,10.559,0.101333,0.175983,4.479,5.333333,921.491,insect_ant,['Ants are one of the most common insects that...
4,Children Learn About The Ant,cXUCUvcscXs,Ant,False,insects live by themselves and have to,welcome to my tree house come and look. at thi...,8.639,12.880,0.143983,0.214667,4.241,5.333333,921.491,insect_ant,['Ants are one of the most common insects that...


### clean the reference txt and mark the manually extracted subtopics

In [12]:
from Python_Functions.NLP_Data_Engineer_Functions import stopwords_generator, lemmatize_stemming, TF_whole_text, TF_IDF_text_doc, topic_extract, \
                          lemmatize_preprocess, text_cleaning_sub, text_cleaning, stop_words

In [13]:

## from the PPT

birds_tailorbird_manual_keywords = ["tailorbird", 'colour', 'color' "appearances",\
                                    "beak", "food", 'voice', 'habitat', 'nest', "egg"  ]

birds_tailorbird_manual_useless_sentences = ['Hey I am Tinky  a Tailorbird. Do you know that I sing melodious songs? Let’s learn more about me!'] 


birds_peacock_manual_keywords = ["peacock", "peafowl", 'feature', 'specialty', 'train', 'crest', 'peahen',
                                 'green peafowl', 'white peafowl', 'colour', 'color' "appearances",\
                                    "beak", "food", 'voice', 'habitat', 'nest', "egg"  ]

birds_peacock_manual_useless_sentences = ['Source: peacocksuk.com', 'Source: pinterest.com', 'Source: natureconservation.in', \
                                         'Source: en.wikipedia.org'] 


birds_flamingo_manual_keywords = ["flamingo", 'feature', 'pink', 'specialty', 'train', 'carotenoids'
                                 'lesser flamingo', 'american flamingo',  'andean flamingo',\
                                 'chilean flamingo' , 'james flamingo', 'colour', 'color' "appearances",\
                                    "beak", "food", 'voice', 'habitat', 'nest', "egg", 'milk'  ]

birds_flamingo_manual_useless_sentences = ['Source: ebird.org', 'Source: en.wikipedia.org, commons.wikimedia.org'] 


insect_ant_manual_keywords = ["ant", 'queen', 'male', "drone", 'worker', 'social', 'princess', 'soldier', 'larva', 'larvae',\
                              'castes', 'black ants', 'appearance', 'habit', 'carpenter', 'weaver', 'leafcutter'\
                              "Odorous house", "food", 'nest', "egg", 'Life cycle'  ]

insect_ant_manual_useless_sentences = ['GLOSSARY: 1.Honeydew: a sweet sticky substance produced by insects while they feed on the plant sap. 2.Plant secretion: the produce of plants such as gums resins etc.'] 


insect_beetle_manual_keywords = ["beetle", 'elytra', 'wing', "ladybug", 'appearance', 'habit', 'scarab', 'mango stem borer', 'weevil', 'dung',\
                              'castes', 'black ants', 'appearance', 'habit', 'carpenter', 'weaver', 'leafcutter'\
                              "Odorous house", "food", 'nest', "egg", 'Life cycle'  ]

insect_beetle_manual_useless_sentences = ['We see many insects around us. Some have wings like mosquitoes, flies, butterflies, etc., while others do not have wings like ants, caterpillars, etc.',\
                                         '\\t', 'GLOSSARY: Species: A group of living organisms that are alike in their features and behaviour. Snout: a long nose. Millet: A type of cereal.' ] 

strings_to_remove = birds_tailorbird_manual_useless_sentences + birds_peacock_manual_useless_sentences + birds_flamingo_manual_useless_sentences +\
             insect_ant_manual_useless_sentences + insect_beetle_manual_useless_sentences

manual_reference_topics = {'bird_tailorbird' :  birds_tailorbird_manual_keywords,
                           'bird_peacock' : birds_peacock_manual_keywords, 
                           'bird_flamingo' : birds_flamingo_manual_keywords,
                          'insect_ant' : insect_ant_manual_keywords, 
                           'insect_beetle' : insect_beetle_manual_keywords}


In [14]:
import re

def remove_urls (vTEXT):
    vTEXT = re.sub(r'(https?:\/\/)(\s)*(www\.)?(\s)*((\w|\s)+\.)*([\w\-\s]+\/)*([\w\-]+)((\?)?[\w\s]*=\s*[\w\%&]*)*', '', vTEXT, flags=re.MULTILINE)
    vTEXT = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', vTEXT, flags=re.MULTILINE)
    vTEXT = vTEXT.replace("'/'","").replace("/","").replace("'https://'", '').replace("'[Music]'", '')
    return(vTEXT)

def form_txt_chunks(txt_lst, sep = ' '):
    
    if txt_lst != []:
        start_txt = ''
        for indx, sen in enumerate(txt_lst):
            if indx == 0:
                put_togather = start_txt + sen 
            else:
                put_togather = put_togather + sep + sen
        return put_togather
    else: return ""
        
def clean_txt_chunk(txt):
    
    sep_lst = txt.split(",")
    cleaned_lst = [i.replace("'", '').strip().replace("[", '').replace("]", '').replace(".jpg", '').replace(".html", '') for i in sep_lst]
    put_togather = form_txt_chunks(txt_lst = cleaned_lst, sep = ' ')

    return remove_urls (vTEXT = put_togather)


In [15]:
refer_txt_df = main_df[['reference_text', 'main_topics']].drop_duplicates().reset_index(drop = True)

refer_txt_df["reference_text_cleaned"] = refer_txt_df['reference_text'].apply(clean_txt_chunk)

# remove the useless sentecnes that could be affect the learning of the concepts/facts
refer_txt_df['reference_text_cleaned_2x'] = refer_txt_df['reference_text_cleaned']
for strg in strings_to_remove:
    refer_txt_df['reference_text_cleaned_2x'] = refer_txt_df['reference_text_cleaned_2x'].str.replace(strg, "")

# apply the manally selected sub-topic key words to the data frame
main_topics_lst = []
manual_topics_string = []
for key, manual_topics in manual_reference_topics.items():
    manual_topics_string.append(form_txt_chunks(txt_lst = manual_topics, sep = ','))
    main_topics_lst.append(key)


df_manual_topics = pd.DataFrame(list(dict(zip(main_topics_lst, manual_topics_string)).items()),columns = ['main_topics','manual_subtopics']) 
refer_txt_df = pd.merge(refer_txt_df, df_manual_topics, "left", 'main_topics')

refer_txt_df_new = refer_txt_df[[ 'reference_text_cleaned_2x', 'manual_subtopics', 'main_topics']]


# form the sentences level from the reference text
refertxt_sentences_lvl_df = pd.DataFrame()
for index, row in refer_txt_df_new.iterrows():
    refertxt_sentences_lvl_df_temp = pd.DataFrame()
    txt = row['reference_text_cleaned_2x']
    clss = row['main_topics']
    manual_subtopic = row['manual_subtopics']
    
    cleaned_temp = text_cleaning(txt = txt, stop_words = stop_words)

    cleaned_words_agg  = form_txt_chunks(txt_lst = cleaned_temp[1], sep = ',')
    vocabulary_agg  = form_txt_chunks(txt_lst =  cleaned_temp[2], sep = ',') # unique words
    tokens_sentences_lst = cleaned_temp[3]
    
    refertxt_sentences_lvl_df_temp['sentences'] = tokens_sentences_lst
    refertxt_sentences_lvl_df_temp['unique_vocabulary_agg'] = vocabulary_agg
    refertxt_sentences_lvl_df_temp['cleaned_words_agg'] = cleaned_words_agg
    refertxt_sentences_lvl_df_temp['reference_text_cleaned']  = txt
    refertxt_sentences_lvl_df_temp['manual_subtopics']  = manual_subtopic
    refertxt_sentences_lvl_df_temp['main_topics'] = clss
    
    refertxt_sentences_lvl_df = pd.concat([refertxt_sentences_lvl_df, refertxt_sentences_lvl_df_temp], axis = 0)
    
    

/var/folders/7g/qq8zpbxn66x6z5czndqlhg640000gn/T/ipykernel_27868/4127312558.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  refer_txt_df['reference_text_cleaned_2x'] = refer_txt_df['reference_text_cleaned_2x'].str.replace(strg, "")


In [16]:
main_df = pd.merge(main_df, refertxt_sentences_lvl_df[['reference_text_cleaned', 'main_topics' ]].drop_duplicates(),\
                   'left', 'main_topics')



In [17]:
main_df.shape

(44368, 16)

In [18]:
refertxt_sentences_lvl_df.to_csv("ready_for_model_dataset/refertxt_sentences_lvl_df.csv")

In [19]:
main_df.to_csv("ready_for_model_dataset/ready_for_model.csv")